In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose
from tsmoothie.smoother import ConvolutionSmoother
from tsmoothie.bootstrap import BootstrappingWrapper
from sklearn.mixture import GaussianMixture as GMM
from sklearn.mixture import BayesianGaussianMixture as BGMM
import pandas as pd
import seaborn as sns; sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import AgglomerativeClustering as hier
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from sklearn.mixture import GaussianMixture as GMM
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import rcParams

warnings.filterwarnings('ignore')


# sns.set()

class Bus:

    def __init__(self, real_d=1, react_d=1, days=365):
        self.days = days
        self.real_d = real_d
        self.react_d = react_d
        if ((self.real_d != 1) & (self.react_d != 1)):
            self.real = self.read(self.real_d, self.days)
            self.react = self.read(self.react_d, self.days)

    # thn kaloume pio panw kai ftiaxnei thn class
    def read(self, directory, days):
        dateparse = lambda j: dt.datetime.strptime(str(j), "%Y%m%d%H%M")

        cells = ['Date', 'Value']
        X = pd.read_excel(directory, usecols=[3, 4], nrows=96 * days, header=None, names=cells,
                          index_col='Date', parse_dates=True, date_parser=dateparse)
        return X

    # dinei stats gia real kai react
    def stats(self):
        mean = self.real['Value'].mean()
        var = self.real['Value'].var()
        mean_r = self.react['Value'].mean()
        var_r = self.react['Value'].var()
        print("mean real = {:.2f}, \nvar real = {:.2f}, \nmean react = {:.2f}, \nvar react = {:.2f}".format(mean, var,
                                                                                                            mean_r,
                                                                                                            var_r))

    # epistrefei ena mhna san DFr, dexetai power = 'real','react','all' kai month
    def month(self, year=2018, month=1, power='real'):
        start = dt.datetime(year, month, 1)
        if (month != 12):
            end = dt.datetime(year, month + 1, 1)
        else:
            end = dt.datetime(year + 1, 1, 1)

        if power == 'real':
            return self.real.iloc[(self.real.index >= start) & (self.real.index < end)]
        elif power == 'react':
            return self.react.iloc[(self.react.index >= start) & (self.react.index < end)]
        elif power == 'all':
            a = self.real.iloc[(self.real.index >= start) & (self.real.index < end)]
            b = self.react.iloc[(self.react.index >= start) & (self.react.index < end)]
            
            return a, b
    def season(self, year=2018, season ='winter', power='real'):
        
        if season == 'winter':
            months =[12,1,2]
        elif season == 'spring':
            months = [3, 4, 5]
        elif season == 'summer':
            months = [6, 7, 8]
        elif season == 'fall':
            months = [9, 10, 11]
        season = pd.DataFrame()
        for month in months:
            season = season.append(self.month(month=month, power=power))
        return season, months
    


# δεχεται το Χ['Value'] και επιστρεφει weekdays,weekends
def workdays(data):
    data.name = 'Value'
    data = pd.DataFrame(data)
    func = lambda x: x.index.weekday + 1
    data['Weekday'] = data.apply(func)

    Weekdays = data['Value'].loc[data['Weekday'] <= 5]
    Weekends = data['Value'].loc[data['Weekday'] > 5]

    return Weekdays, Weekends


     
def choose(howmany, fromwhere, positions):
    import random 
    chose_from = [fromwhere[i] for i in positions]
    chosen = random.sample(chose_from, k=howmany)
    return chosen

def workdayspermonth(month=1,year=2018):
    start = dt.date(year, month, 1)
    if (month != 12):
        end = dt.date(year, month + 1, 1)
    else:
        end = dt.date(year + 1, 1, 1)
    wdays = np.busday_count(start, end)
    wends = (end-start).days-wdays
    return wdays,wends


def unique(list1):
# initialize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

# Οπτικοποίηση δεδομένων με clustering
def visualization(data, days, n_components=10, labels=1, title= 'Generated'):
    
    Y = np.array(data).reshape(-1,96)
    dic = Y[0:days]

    compon = n_components
    model = GMM(n_components=compon,max_iter=1000, random_state=1)
    if labels == 1: labels = [model.fit_predict(dic)]
    

    axis = np.array
    axis = np.arange(96)
    cols = 6
    rows = int(compon//6 + 1) if (compon%6 != 0)  else int(compon//6)

    fig, axs = plt.subplots(ncols=cols, nrows=rows, sharey=True, figsize=(30,12));
    for i, j in enumerate(dic):
        sns.lineplot(x=axis, y=j, ax=axs[labels[0][i]//cols][labels[0][i]%cols])
    
    flattened_axs = np.ravel(axs)
    # Set titles for each subplot in the flattened array
    for i, ax in enumerate(flattened_axs):
        ax.set_title(f'Cluster {i+1}')
    #title of figure
    fig.suptitle(title, fontsize=30)

    
    return labels

def find_occurrence(lista, value, number_of_occurrence):
    count = 0
    for index, item in enumerate(lista):
        if item == value:
            count += 1
            if count == number_of_occurrence:
                return index
    return -1  # Return -1 if the occurrence is not found

def without_extreme_outliers (data, labels, threshold = 3.0):
    from scipy.stats import zscore
    data = np.array(data).reshape(-1,96)
    
    unique_labels = np.unique(labels)
    outlier_points_per_label = {}  # Dictionary to store outlier positions for each label
    zscore_threshold = threshold  # Set the z-score threshold for identifying outliers
    mean_per_labels_column = [] #size (labels,96)

    for label in unique_labels:
        # Step 1: Create a subset of Y for the current label
        part_data = data[labels[0] == label]

        # Step 2: Calculate the mean of each column in part_Y
        column_means = np.mean(part_data, axis=0)
        mean_per_labels_column.append(column_means)

        # Step 3: Calculate the z-scores for each data point in part_Y
        zscores = zscore(part_data, axis=0)

        # Step 4: Find the indices of outliers based on the z-score threshold
        outlier_indices = np.where(np.abs(zscores) > zscore_threshold)

        # Step 5: Save the positions of time series with outliers for the current label
        outlier_points_per_label[label] = outlier_indices
        
        # Antikatastash extreme outliers me meso oro ekeinhs ths sthlhw ana column
        for key in outlier_points_per_label:
            for count, row in enumerate(outlier_points_per_label[key][0]):
                data[find_occurrence(labels[0],key,row+1)][outlier_points_per_label[key][1][count]] = mean_per_labels_column[key][[outlier_points_per_label[key][1][count]]][0]

    return data

#find transition matrix for markov chain if you have a sequence
def trans_matrix (sequence):
    seq = sequence
    return pd.crosstab(pd.Series(seq[:-1],name='State'), pd.Series(seq[1:],name='Next state'), normalize = 'index')

#simulate a sequence if you have a transition matrix
def simul_markov(tran_matr, state_zero=0, lenght = 10):
    sim = []
    #state zero
    state_zero = state_zero
    #next state
    n_state = np.random.choice(tran_matr.iloc[state_zero].index, p = tran_matr.iloc[state_zero])

    while len(sim) < lenght:
        sim.append(n_state)
        n_state = np.random.choice(tran_matr.iloc[tran_matr.index.get_loc(n_state)].index, p = tran_matr.iloc[tran_matr.index.get_loc(n_state)])
    return sim

# pairnei data, way (yearly, monthly), month kai epistrefei T,S,RES, REAL
def decompose_bootstrap(data, year=2018, month=1, period=96):
    fig, axes = plt.subplots(4, 1, figsize=(14, 14))
    fig.tight_layout()
   
    # vriskw start kai end
    start = dt.datetime(year, month, 1)
    if (month != 12):
        end = dt.datetime(year, month + 1, 1)
    else:
        end = dt.datetime(year + 1, 1, 1)

    series = data['Value'].iloc[(data.index >= start) & (data.index < end)]
    results = seasonal_decompose(series, model="aditive", extrapolate_trend=1, period=period)
    trend_estimate = results.trend
    periodic_estimate = results.seasonal
    residual = results.resid

    axes[0].plot(series, label='Original time series', color='blue')
    axes[1].plot(trend_estimate, label='Trend of time series', color='blue')
    axes[2].plot(periodic_estimate, label='Seasonality of time series', color='blue')
    axes[3].plot(residual, label='Decomposition residuals of time series', color='blue')

    for i in axes:
        i.legend(loc='best')

    plt.show()
    plt.close()

    return trend_estimate, periodic_estimate, residual, series

def decompose_bootstrap_year(data, year=2018, period=96):
    fig, axes = plt.subplots(4, 1, figsize=(14, 14))
    fig.tight_layout()
   
    # vriskw start kai end
    start = dt.datetime(2018, 1, 1)
    end = dt.datetime(2019, 1, 1)

    series = data['Value'].iloc[(data.index >= start) & (data.index < end)]
    results = seasonal_decompose(series, model="aditive", extrapolate_trend=1, period=period)
    trend_estimate = results.trend
    periodic_estimate = results.seasonal
    residual = results.resid

    axes[0].plot(series, label='Original time series', color='blue')
    axes[1].plot(trend_estimate, label='Trend of time series', color='blue')
    axes[2].plot(periodic_estimate, label='Seasonality of time series', color='blue')
    axes[3].plot(residual, label='Decomposition residuals of time series', color='blue')

    for i in axes:
        i.legend(loc='best')

    plt.show()

    return trend_estimate, periodic_estimate, residual


# pairnei residuals kai dinei resampled
def bootstrap_res(residual):
    plt.figure(figsize=(20, 20))
    plt.tight_layout()
    data = residual
    axis = np.arange(0, len(data))
    vlines = np.arange(0, len(data), 96)

    # epilogh smoother
    smoother1 = ConvolutionSmoother(window_len=len(data), window_type='ones')

    # epistrofi smoothed data gia plot
    a1 = smoother1.smooth(data)

    # paragwgh samples kai eyresi mean
    bts = BootstrappingWrapper(a1, bootstrap_type='nbb', block_length=96)
    bts_samples = bts.sample(data, n_samples=1)
    mean = bts_samples.mean(axis=0)

    plt.subplot(211)
    # smoothed
    sns.lineplot(x=axis, y=a1.smooth_data[0], linewidth=2, color='green')
    # real
    sns.lineplot(x=axis, y=data, linewidth=1)
    plt.vlines(x=vlines, ymin=min(data), ymax=max(data), colors='red')

    plt.subplot(212)
    plt.vlines(x=vlines, ymin=min(data), ymax=max(data), colors='red')
    # samples
    plt.plot(axis, bts_samples.T, alpha=2, c='orange')

    # mean
    sns.lineplot(x=axis, y=mean, linewidth=1)
    plt.close()

    return mean.tolist()

def concatinate(WD, WE, month=1):
    
    start = dt.datetime(2018, month, 1)
    if (month != 12):
        end = dt.datetime(2018, month + 1, 1)
    else:
        end = dt.datetime(2018 + 1, 1, 1)

    days = pd.date_range(start=start, end=end, freq='1D', closed='left')
    we_iter = 0
    wd_iter = 0
    bs_month = []
    for day in days:
        if (day.weekday() < 5):
            bs_month.extend(WD[wd_iter:(wd_iter + 96)])
            wd_iter += 96
        else:
            bs_month.extend(WE[we_iter:(we_iter + 96)])
            we_iter += 96

    index = pd.date_range(start=start, end=end, freq='15min', closed='left')
    bs_month = pd.Series(bs_month, index=index)
    return bs_month

def concatinate_season(dictmonths, months):
    import collections
    Months = collections.OrderedDict(sorted(dictmonths.items()))
    bs_month = []
    for month in months:
        start = dt.datetime(2018, month, 1)
        if (month != 12):
            end = dt.datetime(2018, month + 1, 1)
        else:
            end = dt.datetime(2018 + 1, 1, 1)
        days = pd.date_range(start=start, end=end, freq='1D', closed='left')
        we_iter = 0
        wd_iter = 0
        for day in days:
            if (day.weekday() < 5):
                bs_month.extend(Months[month][0][wd_iter])
                wd_iter += 1
            else:
                bs_month.extend(Months[month][1][we_iter])
                we_iter += 1

    bs_month = np.array(bs_month)
    bs_month = bs_month.reshape(-1,96)
    return bs_month

In [2]:
def generate_season_2_1(data, season, dict_months, months):
    def unique(list1):
    # initialize a null list
        unique_list = []

        # traverse for all elements
        for x in list1:
            # check if exists in unique_list or not
            if x not in unique_list:
                unique_list.append(x)
        return unique_list
    
    data = data
    season = season
    Months = dict_months
    months = months
    
    # Χωρίζω εργάσιμες και κανονικές
    wd,we = workdays(data)
    wd.shape, we.shape
    wd_np = np.array(wd).reshape(-1,96)
    we_np = np.array(we).reshape(-1,96)

    # Φιτάρω εργασιμες, κανονικες ξεχωριστα και παράγω 200 δείγματα για το καθένα
    modelwd = GMM(n_components=5, random_state=123)
    modelwe = GMM(n_components=3, random_state=123)
    labelswd = modelwd.fit_predict(wd_np)
    labelswe = modelwe.fit_predict(we_np)
    xwd, ywd = modelwd.sample(n_samples=500)
    xwe, ywe = modelwe.sample(n_samples=500)

    # Χωρίζω σε Wds, Wes τοσο τα real οσο και τα generated
    uniquewd = unique(labelswd)
    uniquewe = unique(labelswe)

    dictywd = {}
    for unique in uniquewd:
        dictywd[unique] = [i for i,j in enumerate(ywd) if j==unique]

    dictrwd = {}
    for unique in uniquewd:
        dictrwd[unique] = [i for i,j in enumerate(labelswd) if j==unique]

    dictrwe = {}
    for unique in uniquewe:
        dictrwe[unique] = [i for i,j in enumerate(labelswe) if j==unique]

    dictywe = {}
    for unique in uniquewe:
        dictywe[unique] = [i for i,j in enumerate(ywe) if j==unique]

    
    # Κανω δειγματοληψία απο τα generated για το σύνολο των WDs, WEs της εποχής, 
    #οπότε έχω δύο arrays με τόσα generated όσα οι WEs, WDs της εποχής αντίστοιχα

    genwd = np.array(wd_np)
    for cluster in uniquewd:
        chosen = choose(len(dictrwd[cluster]),xwd,dictywd[cluster])
        for i,k in zip(dictrwd[cluster],chosen):
            genwd[i] = k.round(2)

    genwe = np.array(we_np)
    for cluster in uniquewe:
        chosen = choose(len(dictrwe[cluster]),xwe,dictywe[cluster])
        for i,k in zip(dictrwe[cluster],chosen):
            genwe[i] = k.round(2)

    sumwd = sumwe = 0
    for month in months:
        wdays, wends = workdayspermonth(month=month)
        Months[month] = (genwd[sumwd:sumwd+wdays],genwe[sumwe:sumwe+wends])
        sumwd += wdays
        sumwe += wends
        
    return Months

def generate_season_2_2(data, season, dict_months, months):
    def unique(list1):
    # initialize a null list
        unique_list = []

        # traverse for all elements
        for x in list1:
            # check if exists in unique_list or not
            if x not in unique_list:
                unique_list.append(x)
        return unique_list
    
    data = data
    season = season
    Months = dict_months
    months = months
    
    # Χωρίζω εργάσιμες και κανονικές
    wd,we = workdays(data)
    wd.shape, we.shape
    wd_np = np.array(wd).reshape(-1,96)
    we_np = np.array(we).reshape(-1,96)

    # Φιτάρω εργασιμες, κανονικες ξεχωριστα και παράγω 200 δείγματα για το καθένα
    modelwd = GMM(n_components=5, random_state=123)
    modelwe = GMM(n_components=3, random_state=123)
    labelswd = modelwd.fit_predict(wd_np)
    labelswe = modelwe.fit_predict(we_np)
    xwd, ywd = modelwd.sample(n_samples=800)
    xwe, ywe = modelwe.sample(n_samples=800)

    # Βρισκω trans Matrix για labels των real δεδομενων
    transM_wd = trans_matrix(labelswd)
    transM_we = trans_matrix(labelswe)

    # Κανω simulate μια λιστα με βαση τον προηγουμενο trans matrix
    simul_labels_wd = simul_markov(transM_wd, state_zero=labelswd[0], lenght=len(labelswd))
    simul_labels_we = simul_markov(transM_we, state_zero=labelswe[0], lenght=len(labelswe))

    uniquewd = unique(simul_labels_wd)
    uniquewe = unique(simul_labels_we)


    dictywd = {}
    for unique in uniquewd:
        dictywd[unique] = [i for i,j in enumerate(ywd) if j==unique]

    dictywe = {}
    for unique in uniquewe:
        dictywe[unique] = [i for i,j in enumerate(ywe) if j==unique]

    dictswd = {}
    for unique in uniquewd:
        dictswd[unique] = [i for i,j in enumerate(simul_labels_wd) if j==unique]

    dictswe = {}
    for unique in uniquewe:
        dictswe[unique] = [i for i,j in enumerate(simul_labels_we) if j==unique]
    
    # Κανω δειγματοληψία απο τα generated για το σύνολο των WDs, WEs της εποχής, 
    #οπότε έχω δύο arrays με τόσα generated όσα οι WEs, WDs της εποχής αντίστοιχα

    genwd = np.array(wd_np)
    for cluster in uniquewd:
        chosen = choose(len(dictswd[cluster]),xwd,dictywd[cluster])
        for i,k in zip(dictswd[cluster],chosen):
            genwd[i] = k.round(2)

    genwe = np.array(we_np)
    for cluster in uniquewe:
        chosen = choose(len(dictswe[cluster]),xwe,dictywe[cluster])
        for i,k in zip(dictswe[cluster],chosen):
            genwe[i] = k.round(2)

    sumwd = sumwe = 0
    for month in months:
        wdays, wends = workdayspermonth(month=month)
        Months[month] = (genwd[sumwd:sumwd+wdays],genwe[sumwe:sumwe+wends])
        sumwd += wdays
        sumwe += wends
        
    return Months

def generate_year_2_3(bus, period, power = 'real'):
    year = pd.DataFrame()
    for month in range(1,13):
        t,s,res,real = decompose_bootstrap(bus.month(month=month, power=power), month=month, period=period)
        weekdays, weekends = workdays(res)
        resampled_wd = bootstrap_res(weekdays)
        resampled_we = bootstrap_res(weekends)
        resampled_month_res = concatinate(resampled_wd, resampled_we,month=month)

        mhnas = dt.datetime.strptime(str(month), "%m").strftime("%B")
        #plt.figure(figsize=(20, 20))
        #plt.tight_layout()

        generated = t+s+resampled_month_res
        generated.name ='Value'
        gen = pd.DataFrame(generated)

        real = pd.DataFrame(real)
        year = year.append(gen)

        plt.subplot(211)
        plt.title(mhnas)
        plt.plot(real['Value'], label='Real time series')
        plt.plot(real['Value'])
        plt.legend(loc='best')

        plt.subplot(212)
        plt.plot(gen['Value'], label='Generated time series')
        plt.plot(gen['Value'])
        plt.legend(loc='best')
    
    return year
